In [ ]:
# x_4Capstocks.py

"""Program to prepare BUY trades for Capstocks
Creates a spreadsheet called capstocks.xlsx that can be sent to Senjith
Date: 23-July-2019
Ver: 1.0
Time taken: milliseconds
"""

from z_helper import *
util.startLoop()

# from json
a = assign_var('nse') + assign_var('common')
for v in a:
    exec(v)

def capstocks(cap_blacklist):
    '''prepares BUY trade spreadsheet for Capstocks - for YesBank+Kashi - NRE
    Args:
        (cap_blacklist) as list of scrips already held. From Xn googlespreadsheet
    Dependancies: ohlcs.pkl and targets.pkl
    Returns:
        (df) as DataFrame of capstocks trades'''
    
    # get the blacklist from google spreadsheet
    askmsg="\nCopy Xn -> Xn [A:P] to clipboard and press Enter to continue...\n"
    input(askmsg)
    df=pd.read_clipboard()

    # make qty and margin numeric
    df=df.assign(margin=pd.to_numeric(df['Total Cost'].str.replace(',','')), qty=pd.to_numeric(df['Units'].str.replace(',','')))

    # groupby and sort down by margin
    df=df.groupby('Stock').sum().sort_values('margin', ascending=False)

    # stocks to be blacklisted
    cb=json.dumps(list(df[df.margin>200000].reset_index().Stock))
    cap_blacklist=eval(cb) # generate the blacklist
    print(f"\ncopy...\n{cb}\n...to variables.json[cap_blacklist]\n")
    
    # Blacklist from Google Xn
    cap_blacklist = {c[:9] for c in cap_blacklist}

    # get the ohlcs
    df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')
    df_targets = pd.read_pickle(fspath+'targets.pkl')

    # remove blacklists from targets
    df_targets = df_targets[~df_targets.symbol.isin(cap_blacklist)]

    #... get 50% of max 1-day Standard Deviation

    # get max fall rise for 1-day Standard deviation
    df_ohlc = df_ohlcs.assign(OneDayVar = abs(df_ohlcs[['symbol', 'close']].groupby('symbol').agg('diff')))

    df_onedayvar = df_ohlc[['symbol', 'OneDayVar']].groupby('symbol').agg('max')

    df = df_targets.drop_duplicates('symbol').set_index('symbol').join(df_onedayvar).reset_index()

    cols = ['symbol', 'undId', 'lot', 'undPrice', 'OneDayVar', 'lo52', 'hi52']
    df1 = df[cols]

    df1 = df1.assign(expPrice3 = df1.undPrice-(df1.OneDayVar/3).apply(lambda x: get_prec(x, prec)), 
                     expPrice2 = df1.undPrice-(df1.OneDayVar/2).apply(lambda x: get_prec(x, prec)), 
                     expPrice1 = df1.undPrice-(df1.OneDayVar/1.5).apply(lambda x: get_prec(x, prec)))

    # get the deltas against hi52 and lo52
    df2 = df1.assign(lodelta=df1.undPrice/df1.lo52, hidelta=df1.hi52/df1.undPrice).sort_values('lodelta')

    # filter out target buys
    df3 = df2[(df2.lodelta < df2.lodelta.mean()) & (df2.hidelta > df2.hidelta.mean())]

    df4 = pd.melt(df3[['symbol', 'lot', 'undPrice', 'expPrice1', 'expPrice2', 'expPrice3']], 
                    id_vars=['symbol','lot', 'undPrice'],
                    value_vars=['expPrice1', 'expPrice2', 'expPrice3'],
                    value_name='LimitPrice',
                    var_name='variable')

    # Remove negative limitPrice
    df5 = df4[df4.LimitPrice > 0].reset_index(drop=True).sort_values('symbol')

    # Get the multiple of lots to be bought
    df6 = df5.assign(mult=df5.variable.str[-1:].astype('int32'))
    
    # Divide the multiple to appropriately adjust to remaining balance in YesBank NRE
    df7=df6.assign(qty=(df6.lot/df6.mult/capstocksmult).astype('int').apply(lambda x: int(10 * round(float(x)/10))), trade='BUY')
    
    df7.loc[df7.lot <= 30, 'qty'] = 1 # Make the quantity to be 3 for very high margin symbols [EICHERMOT, PAGEIND]

    df7 = df7.sort_values(['symbol', 'LimitPrice'], ascending=[True, False])

    df7[['symbol', 'trade', 'LimitPrice', 'qty']].to_excel(fspath+'capstocks.xlsx', freeze_panes=(1,1), index=False)

    # make a watchlist to see Analyst Recommendations. Use only those which are BUY, OUTPERFORM.
    watch = [('DES', s, 'STK', 'NSE') for s in df7.symbol.unique()]
    util.df(watch).to_csv(fspath+'watch.csv', header=None, index=None)
    
    return df7

In [1]:
# Blacklist from Google Xn
cap_blacklist = 'ASHOKLEY,BHARATFORG,GRASIM,PETRONET,SUNTV,ICICIPRULI,ARVIND,BSOFT,ENGINERSIN,TATAELXSI,MOTHERSUMI,TVSMOTOR'.split(',')
cap_blacklist = {c[:9] for c in cap_blacklist}

from z_helper import *
util.startLoop()

# from json
a = assign_var('nse') + assign_var('common')
for v in a:
    exec(v)


    
# Blacklist from Google Xn
cap_blacklist = {c[:9] for c in cap_blacklist}

# get the ohlcs
df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')
df_targets = pd.read_pickle(fspath+'targets.pkl')

# remove blacklists from targets
df_targets = df_targets[~df_targets.symbol.isin(cap_blacklist)]

#... get 50% of max 1-day Standard Deviation

# get max fall rise for 1-day Standard deviation
df_ohlc = df_ohlcs.assign(OneDayVar = abs(df_ohlcs[['symbol', 'close']].groupby('symbol').agg('diff')))

df_onedayvar = df_ohlc[['symbol', 'OneDayVar']].groupby('symbol').agg('max')

df = df_targets.drop_duplicates('symbol').set_index('symbol').join(df_onedayvar).reset_index()

cols = ['symbol', 'undId', 'lot', 'undPrice', 'OneDayVar', 'lo52', 'hi52']
df1 = df[cols]

df1 = df1.assign(expPrice3 = df1.undPrice-(df1.OneDayVar/3).apply(lambda x: get_prec(x, prec)), 
                 expPrice2 = df1.undPrice-(df1.OneDayVar/2).apply(lambda x: get_prec(x, prec)), 
                 expPrice1 = df1.undPrice-(df1.OneDayVar/1.5).apply(lambda x: get_prec(x, prec)))

# get the deltas against hi52 and lo52
df2 = df1.assign(lodelta=df1.undPrice/df1.lo52, hidelta=df1.hi52/df1.undPrice).sort_values('lodelta')

# filter out target buys
df3 = df2[(df2.lodelta < df2.lodelta.mean()) & (df2.hidelta > df2.hidelta.mean())]

df4 = pd.melt(df3[['symbol', 'lot', 'undPrice', 'expPrice1', 'expPrice2', 'expPrice3']], 
                id_vars=['symbol','lot', 'undPrice'],
                value_vars=['expPrice1', 'expPrice2', 'expPrice3'],
                value_name='LimitPrice',
                var_name='variable')

# Remove negative limitPrice
df5 = df4[df4.LimitPrice > 0].reset_index(drop=True).sort_values('symbol')

# Get the multiple of lots to be bought
df6 = df5.assign(mult=df5.variable.str[-1:].astype('int32'))

# Divide the multiple to appropriately adjust to remaining balance in YesBank NRE
df7=df6.assign(qty=(df6.lot/df6.mult/capstocksmult).astype('int').apply(lambda x: int(10 * round(float(x)/10))), trade='BUY')

df7.loc[df7.lot <= 30, 'qty'] = 1 # Make the quantity to be 3 for very high margin symbols [EICHERMOT, PAGEIND]

df7 = df7.sort_values(['symbol', 'LimitPrice'], ascending=[True, False])

df7[['symbol', 'trade', 'LimitPrice', 'qty']].to_excel(fspath+'capstocks.xlsx', freeze_panes=(1,1), index=False)

# make a watchlist to see Analyst Recommendations. Use only those which are BUY, OUTPERFORM.
watch = [('DES', s, 'STK', 'NSE') for s in df7.symbol.unique()]
util.df(watch).to_csv(fspath+'watch.csv', header=None, index=None)

In [3]:
df7.head()

,symbol,lot,undPrice,variable,LimitPrice,mult,qty,trade
52,APOLLOTYR,3000,154.35,expPrice3,149.30,3,200,BUY
29,APOLLOTYR,3000,154.35,expPrice2,146.75,2,300,BUY
8,APOLLOTYR,3000,154.35,expPrice1,144.20,1,600,BUY
46,BALKRISIN,800,716.30,expPrice3,674.65,3,50,BUY
23,BALKRISIN,800,716.30,expPrice2,653.85,2,80,BUY


In [4]:
df.assign(margin=df7.LimitPrice*df7.qty).groupby('symbol').sum().sort_values('margin', ascending=False).head()

,optId,dte,lot,undPrice,strike,stDev,lo52,hi52,margin,qty,close,rom,sd,expPrice,OneDayVar
symbol,,,,,,,,,,,,,,,
TORNTPOWE,373164921,58,3000,306.70,245.0,27.825580,214.00,311.00,104196.0,1,1.5,0.179219,2.217384,6.70,30.00
TITAN,369085534,30,750,1078.90,840.0,105.167775,749.50,1334.70,102120.0,1,1.7,0.111224,2.271608,12.15,153.50
RBLBANK,375784995,30,1200,449.45,270.0,76.828570,398.70,701.45,101288.0,1,0.0,0.015706,2.335720,4.80,79.35
POWERGRID,373177125,58,4000,207.90,242.5,9.772771,174.95,213.85,99540.0,1,0.0,0.016411,3.540449,4.55,8.05
ADANIPOWE,373120881,30,20000,61.60,44.0,6.479384,15.45,67.15,97947.0,1,0.2,0.151876,2.716308,1.50,9.10
